<a href="https://colab.research.google.com/github/weedge/doraemon-nb/blob/main/achatbot_freeze_omni_voice_bot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# install

In [ ]:
!cd /content && rm -rf achatbot && git clone --recursive https://github.com/ai-bot-pro/achatbot.git

In [2]:
%cd /content/achatbot


/content/achatbot


In [ ]:
!bash scripts/pypi_achatbot.sh dev

In [ ]:
!pip install -q "dist/achatbot-0.0.8.3-py3-none-any.whl[fastapi_bot_server,daily_room_audio_stream,livekit_room_audio_stream,agora_channel_audio_stream,silero_vad_analyzer,freeze_omni_voice_processor,queue]"


# download model

In [5]:
from google.colab import userdata
#NGROK_TOKEN=userdata.get('NGROK_TOKEN')
HF_TOKEN=userdata.get('HF_TOKEN')


In [ ]:
!huggingface-cli login --token $HF_TOKEN --add-to-git-credential

In [ ]:
# FreezeOmni ckpt
# - audiollm: encoder and adapter
# - decoder: NAR decoder, AR decoder and codec decoder
!huggingface-cli download VITA-MLLM/Freeze-Omni --local-dir /content/ckpts/VITA-MLLM/Freeze-Omni --local-dir-use-symlinks False

In [ ]:
# textual llm Qwen2-7B-Instruct ckpt
!huggingface-cli download Qwen/Qwen2-7B-Instruct --local-dir /content/ckpts/Qwen/Qwen2-7B-Instruct --local-dir-use-symlinks False

# run webrtc bot

In [9]:
from google.colab import userdata
DAILY_API_KEY=userdata.get('DAILY_API_KEY')

LIVEKIT_URL=userdata.get('LIVEKIT_URL')
LIVEKIT_API_KEY=userdata.get('LIVEKIT_API_KEY')
LIVEKIT_API_SECRET=userdata.get('LIVEKIT_API_SECRET')


In [10]:
%cd /content/achatbot

/content/achatbot


In [13]:
!cat /content/daily_freeze_omni_voice_bot.json

{
  "chat_bot_name": "DailyFreezeOmniVoiceBot",
  "room_name": "chat-room",
  "room_url": "",
  "token": "",
  "room_manager": {
    "tag": "daily_room",
    "args": {
      "privacy": "public"
    }
  },
  "services": {
    "pipeline": "achatbot",
    "vad": "silero",
    "voice_llm": "freeze_omni"
  },
  "config": {
    "vad": {
      "tag": "silero_vad_analyzer",
      "args": { "stop_secs": 0.7 }
    },
    "voice_llm": {
      "tag": "freeze_omni_voice_processor",
      "args": {
        "args": {
          "llm_path": "/content/ckpts/Qwen/Qwen2-7B-Instruct",
          "model_path": "/content/ckpts/VITA-MLLM/Freeze-Omni/checkpoints"
        }
      }
    }
  },
  "config_list": []
}


In [23]:
!DAILY_API_KEY=$DAILY_API_KEY \
  python -m src.cmd.bots.main -f /content/daily_freeze_omni_voice_bot.json

{
    "chat_bot_name": "DailyFreezeOmniVoiceBot",
    "config": {
        "vad": {
            "args": {
                "stop_secs": 0.7
            },
            "tag": "silero_vad_analyzer"
        },
        "voice_llm": {
            "args": {
                "args": {
                    "llm_path": "/content/ckpts/Qwen/Qwen2-7B-Instruct",
                    "model_path": "/content/ckpts/VITA-MLLM/Freeze-Omni/checkpoints"
                }
            },
            "tag": "freeze_omni_voice_processor"
        }
    },
    "config_list": [],
    "room_manager": {
        "args": {
            "privacy": "public"
        },
        "tag": "daily_room"
    },
    "room_name": "chat-room",
    "room_url": "",
    "services": {
        "pipeline": "achatbot",
        "vad": "silero",
        "voice_llm": "freeze_omni"
    },
    "token": ""
}
2024-12-20 02:29:46,830 - chat-bot - INFO - /content/achatbot/src/cmd/bots/main.py:63 - <module> - bot_config:{'chat_bot_name': 'DailyFreezeO

In [ ]:
!ls /usr/local/lib/python3.10/dist-packages/apipeline/

In [ ]:
from IPython.display import Audio
Audio(filename="/content/achatbot/records/bot_speak_2024-12-19_04-51-40.318.wav")


# inference

In [26]:
!cd /content/achatbot/deps/FreezeOmni/ \
  && PYTHONPATH=./:$PYTHONPATH CUDA_VISIBLE_DEVICES=0 python bin/inference.py \
    --model_path /content/ckpts/VITA-MLLM/Freeze-Omni/checkpoints \
    --input_wav /content/hi.wav \
    --output_wav /content/answer_hi.wav \
    --llm_path /content/ckpts/Qwen/Qwen2-7B-Instruct \
    --top_p 0.8 \
    --top_k 20 \
    --temperature 0.8

[2024-12-20 02:54:01.890] Namespace(model_path='/content/ckpts/VITA-MLLM/Freeze-Omni/checkpoints', llm_path='/content/ckpts/Qwen/Qwen2-7B-Instruct', top_k=20, top_p=0.8, temperature=0.8, input_wav='/content/hi.wav', output_wav='/content/answer_hi.wav')
[2024-12-20 02:54:04.775] the number of speech encoder params: 341.3681640625M
Loading checkpoint shards: 100% 4/4 [00:00<00:00, 15.03it/s]
[2024-12-20 02:54:05.674] the number of audio llm params: 7621.362796783447M
[2024-12-20 02:54:05.701] Checkpoint: loading from checkpoint /content/ckpts/VITA-MLLM/Freeze-Omni/checkpoints/audiollm/final.pt for GPU
[2024-12-20 02:54:11.160] reading a config file from /content/ckpts/VITA-MLLM/Freeze-Omni/checkpoints/decoder/model.json
[2024-12-20 02:54:13.306] the number of LLM2TTSCodecAR(NAR decoder and AR decoder(llama transformer blocks)) params: 157.08826065063477M
/usr/local/lib/python3.10/dist-packages/torch/nn/utils/weight_norm.py:28: UserWarning: torch.nn.utils.weight_norm is deprecated in favo

In [29]:
from IPython.display import Audio
Audio(filename="/content/answer_hi.wav")


# inference stream

In [27]:
!cd /content/achatbot/deps/FreezeOmni/ \
  && PYTHONPATH=./:$PYTHONPATH CUDA_VISIBLE_DEVICES=0 python bin/inference_stream.py \
    --model_path /content/ckpts/VITA-MLLM/Freeze-Omni/checkpoints \
    --input_wav /content/hi.wav \
    --output_wav /content/answer_hi_stream.wav \
    --llm_path /content/ckpts/Qwen/Qwen2-7B-Instruct \
    --top_p 0.8 \
    --top_k 20 \
    --temperature 0.8

[2024-12-20 02:54:30.361] Namespace(model_path='/content/ckpts/VITA-MLLM/Freeze-Omni/checkpoints', llm_path='/content/ckpts/Qwen/Qwen2-7B-Instruct', top_k=20, top_p=0.8, temperature=0.8, input_wav='/content/hi.wav', output_wav='/content/answer_hi_stream.wav')
[2024-12-20 02:54:33.236] the number of speech encoder params: 341.3681640625M
Loading checkpoint shards: 100% 4/4 [00:00<00:00, 16.39it/s]
[2024-12-20 02:54:34.125] the number of audio llm params: 7621.362796783447M
[2024-12-20 02:54:34.151] Checkpoint: loading from checkpoint /content/ckpts/VITA-MLLM/Freeze-Omni/checkpoints/audiollm/final.pt for GPU
[2024-12-20 02:54:39.624] reading a config file from /content/ckpts/VITA-MLLM/Freeze-Omni/checkpoints/decoder/model.json
[2024-12-20 02:54:41.768] the number of LLM2TTSCodecAR(NAR decoder and AR decoder(llama transformer blocks)) params: 157.08826065063477M
/usr/local/lib/python3.10/dist-packages/torch/nn/utils/weight_norm.py:28: UserWarning: torch.nn.utils.weight_norm is deprecated 

In [28]:
from IPython.display import Audio
Audio(filename="/content/answer_hi_stream.wav")


# server

In [ ]:
from google.colab import userdata
NGROK_TOKEN=userdata.get('NGROK_TOKEN')
#HF_TOKEN=userdata.get('HF_TOKEN')


In [ ]:
!pip install -q flask_socketio pyngrok

In [ ]:
!ngrok config add-authtoken $NGROK_TOKEN

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [ ]:
!cd /content/achatbot/deps/FreezeOmni/ \
  && PYTHONPATH=./:$PYTHONPATH PYTHONDONTWRITEBYTECODE=1 CUDA_VISIBLE_DEVICES=0 python bin/server.py \
    --ip 127.0.0.1 \
    --port 8000 \
    --max_users 3 \
    --llm_exec_nums 1 \
    --timeout 180 \
    --model_path /content/ckpts/VITA-MLLM/Freeze-Omni/checkpoints \
    --llm_path /content/ckpts/Qwen/Qwen2-7B-Instruct \
    --top_p 0.8 \
    --top_k 20 \
    --temperature 0.8 \
    --ngrok

Namespace(model_path='/content/ckpts/VITA-MLLM/Freeze-Omni/checkpoints', llm_path='/content/ckpts/Qwen/Qwen2-7B-Instruct', top_k=20, top_p=0.8, temperature=0.8, ip='127.0.0.1', port='8000', max_users=3, llm_exec_nums=1, timeout=180, ngrok=True, ssl=False)
[2024-12-19 03:06:10.872] the number of speech encoder params: 341.3681640625M
Loading checkpoint shards: 100% 4/4 [00:00<00:00, 11.21it/s]
[2024-12-19 03:06:11.954] the number of audio llm params: 7621.362796783447M
[2024-12-19 03:06:11.981] Checkpoint: loading from checkpoint /content/ckpts/VITA-MLLM/Freeze-Omni/checkpoints/audiollm/final.pt for GPU
[2024-12-19 03:06:17.588] reading a config file from /content/ckpts/VITA-MLLM/Freeze-Omni/checkpoints/decoder/model.json
[2024-12-19 03:06:17.589] reading a config file from /content/ckpts/VITA-MLLM/Freeze-Omni/checkpoints/decoder/model.json
[2024-12-19 03:06:17.590] reading a config file from /content/ckpts/VITA-MLLM/Freeze-Omni/checkpoints/decoder/model.json
[2024-12-19 03:06:23.026] t

# 总结
- freezeOmni voice 不支持 asr识别出文本，将audio直接encode, 然后进过adapter成对应的token给 llm
- 注意在decoder生成语音的时候，使用asyncio在同一个线程中处理时，decoder是cpu/gpu binding，不会让出cpu, 会导致生成语音需要等到当论会话完成时在播放生成的语音任务得不到调度执行, 解决办法使用单独的线程并发执行,比如：使用to_thread方法，将cpu->gpu的算子kernel任务使用单独的线程来执行，这里freezeOmni voice 主要是两个封装好的函数：
  ```python
  self._outputs = self.inference_pipeline.speech_dialogue(None, **self._outputs)

  # Note: llm2tts.run返回的是迭代器，在运行的时候才会调用，
  # 如果使用to_thread直接裹一层返回的是迭代器，运行时还是在调用iter时执行，
  # 所以需要封装一层，将iter中的数据放入数组中返回
  self.llm2tts.run(
            embeddings.reshape(-1, 896).unsqueeze(0),
            self._args.decoder_top_k,
            hidden_state_output.reshape(-1, 896).unsqueeze(0),
            codec_chunk_size=self._args.codec_chunk_size,
            codec_padding_size=self._args.codec_padding_size,
            penalty_window_size=self._args.decoder_penalty_window_size,
            penalty=self._args.decoder_penalty,
            N=self._args.decoder_N,
            seg_threshold=self._args.decoder_seg_threshold,
            max_tokens=self._args.decoder_max_tokens,
        )

    segs = await asyncio.to_thread(self.decoder, cur_hidden_state, cur_text)

    def decoder(self, cur_hidden_state, cur_text) -> list[torch.Tensor]:
        segs = []
        for seg in self.decoder_iter(cur_hidden_state, cur_text):
            segs.append(seg)
        return segs

    def decoder_iter(self, cur_hidden_state, cur_text) -> Generator[torch.Tensor, None, None]:
        """
        Decodes the current hidden state and text to generate audio segments using speech decoder.
        """
        hidden_state_output = torch.cat(cur_hidden_state).squeeze(1)
        cur_text_procced = self.inference_pipeline.post_process(cur_text)
        logging.info(f"Synthesis: {cur_text_procced}")
        embeddings = self.inference_pipeline.model.llm_decoder.model.embed_tokens(
            torch.tensor(self.inference_pipeline.model.tokenizer.encode(cur_text_procced)).cuda()
        )

        for seg in self.llm2tts.run(
            embeddings.reshape(-1, 896).unsqueeze(0),
            self._args.decoder_top_k,
            hidden_state_output.reshape(-1, 896).unsqueeze(0),
            codec_chunk_size=self._args.codec_chunk_size,
            codec_padding_size=self._args.codec_padding_size,
            penalty_window_size=self._args.decoder_penalty_window_size,
            penalty=self._args.decoder_penalty,
            N=self._args.decoder_N,
            seg_threshold=self._args.decoder_seg_threshold,
            max_tokens=self._args.decoder_max_tokens,
        ):
            yield seg
  # 或者单独启动一个线程循环，使用两个in/out队列, 这样可以seg粒度来吐出audio数据。
  # 具体细节可以参考 bin/inference_stream.py
  ```
- 中断处理，重置，改变speech_dialogue的状态重新开始生成。
- 通过llm生成的某段文本来生成语音片段， 语音片段没有对应的这段文本中的部分文本。需要将这段文本切成刚好只生成这一段语音片段。